In [2]:
import numpy as N
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from datetime import datetime,timedelta
from matplotlib.dates import *
import pytz as pytz
import sys
import os
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
import pyPIPS.disdrometer_module as dis
import pyPIPS.radarmodule as radar
#import obanmodule as oban
import pandas as pd
import glob
import pyart as pyart
from metpy.calc import get_wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
%matplotlib inline
%load_ext autoreload
%autoreload 2


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/Users/ddawson/miniconda3/envs/pyPIPS/lib/python3.6/site-packages/pyart/graph/cm.py:104: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:
/Users/ddawson/miniconda3/envs/pyPIPS/lib/python3.6/site-packages/pyart/graph/cm_colorblind.py:32: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:


In [ ]:
radardir = '/Volumes/depot/dawson29/data/VORTEXSE/obsdata/2017/SMARTR/IOP1A/CFRadial/PPI'
#radfname = 'cfrad.20170325_180005.062_to_20170325_180027.534_CPOL_v1_s01_el0.80_PPI.nc'
radfname = 'cfrad.20170325_180805.033_to_20170325_180817.021_CPOL_v1_s01_el0.80_PPI.nc'

radpathlist = glob.glob(radardir+'/*')

#radpath = os.path.join(radardir,radfname)

fieldnames=['dBZ']
el_req = 0.8
radlat = None
radlon = None
radalt = None

radarsweeplist = []
sweeptimelist = []

for radpath in radpathlist:
    sweeptime = radar._getsweeptime(radpath)
    
    outfieldnames,radarsweep = radar.readCFRadial_pyART(False,0.8,radlat,radlon,radalt,radpath
                                                        ,None,fieldnames,compute_kdp=False)
    radarsweeplist.append(radarsweep)
    sweeptimelist.append(sweeptime)

In [ ]:
# Read in the disdrometer data file
dis_dir = '/Users/dawson29/Dropbox/Projects/VORTEXSE/PIPS_data/2017/IOP1A'
disfilenames = ['PIPS1A_IOP1A_D1.txt','PIPS1B_IOP1A_D1.txt','PIPS2B_IOP1A_D1.txt']
#disfilepath = os.path.join(dis_dir,dis_filename)

windspdlist = []
windspdavgveclist = []
winddirabslist = []
winddiravgveclist = []
fasttemplist = []
dewpointlist = []
pressurelist = []

for disfilename in disfilenames:

    #starttime = '201703251808' # '201703251800'
    #stoptime = '201703251808' # '201703251800'
    
    disfilepath = os.path.join(dis_dir,disfilename)
    datetimesUTC,windspds,winddirabss,fasttemps,dewpoints,pressures,    \
                GPS_lats,GPS_lons,GPS_stats,GPS_alts = dis.readPIPSstation(disfilepath)

    # datetimesUTC,pdatetimesUTC,flaggedtimes,hailflag,intensities,preciptots,reflectivities,pcounts,pcounts2, \
    #            sensortemps,concentrations,onedrop_concentrations,countsMatrix,amplitudes,windspds,winddirrels,winddirabss, \
    #         winddiags,fasttemps,slowtemps,dewpoints,RHs_derived,RHs,pressures,compass_dirs,    \
    #         GPS_lats,GPS_lons,GPS_stats,GPS_alts,voltages,DSD_interval,DSD_intervalstr,DSD_index = \
    #         dis.readPIPS(disfilepath,basicqc=True,rainfallqc=True,rainonlyqc=True,
    #         strongwindqc=True,DSD_interval=60.0)

    # Convert to numpy arrays
    windspds = N.array(windspds)
    winddirabss = N.array(winddirabss)
    fasttemps = N.array(fasttemps)
    dewpoints = N.array(dewpoints)
    pressures = N.array(pressures)
    
    # Plot wind meteogram
    windavgintv = 60
    windgustintv = 3
    
    # Compute wind speed and direction, and wind gusts
    windspdsavg,windspdsavgvec,winddirsavgvec,windgusts,windgustsavg = dis.avgwind(winddirabss,
            windspds,windavgintv,gusts=True,gustintv=windgustintv,center=False)
#     offset = 0
#     windspdsavg,windspdsavgvec,winddirsavgvec,winddirsunitavgvec,windgusts,windgustsavg,usavg,vsavg,unit_usavg,unit_vsavg = \
#     dis.resamplewind(datetimesUTC,offset,winddirabss,windspds,'60S',gusts=True,gustintvstr='3S',center=False)
    
    # Compute potential temperature, water vapor mixing ratio, and density
    # pt = thermo.caltheta(pressures*100.,slowtemps+273.15)
    # qv = thermo.calqv(RHs/100.,pressures*100.,slowtemps+273.15)
    # rho=thermo.calrho(pressures*100.,pt,qv)

    # Determine start and end times/indices for analysis

    datetimesUTCnums = date2num(datetimesUTC)
    
    windspdtlist = []
    windspdavgvectlist = []
    winddirabstlist = []
    winddiravgvectlist = []
    fasttemptlist = []
    dewpointtlist = []
    pressuretlist = []
    
    for sweeptime in sweeptimelist:
        starttime = date2num(sweeptime)
        stoptime = date2num(sweeptime)
        try:
            startindex = next(i for i,t in enumerate(datetimesUTCnums) if t >= starttime)
        except:
            startindex = 0
            starttime = datetimesUTCnums[startindex]
        try:
            stopindex = next(i for i,t in enumerate(datetimesUTCnums) if t >= stoptime)
        except:
            stopindex = N.size(datetimesUTCnums)-1
            stoptime = datetimesUTCnums[stopindex]
    
#     if(N.int(starttime) == -1):
#         startindex = 0
#         starttime = datetimesUTCnums[startindex]
#         plotstarttime = starttime
#     else:
#         starttime = date2num(datetime(N.int(starttime[:4]),N.int(starttime[4:6]),
#                         N.int(starttime[6:8]),N.int(starttime[8:10]),N.int(starttime[10:12])))
#         plotstarttime = starttime
#         try:
#             startindex = next(i for i,t in enumerate(datetimesUTCnums) if t >= starttime)
#         except:
#             startindex = 0
#             starttime = datetimesUTCnums[startindex]

#     if(N.int(stoptime) == -1):
#         stopindex = N.size(datetimesUTCnums)-1
#         stoptime = datetimesUTCnums[stopindex]
#         plotstoptime = stoptime
#     else:
#         stoptime = date2num(datetime(N.int(stoptime[:4]),N.int(stoptime[4:6]),
#                         N.int(stoptime[6:8]),N.int(stoptime[8:10]),N.int(stoptime[10:12])))
#         plotstoptime = stoptime
#         try:
#             stopindex = next(i for i,t in enumerate(datetimesUTCnums) if t >= stoptime)
#         except:
#             stopindex = N.size(datetimesUTCnums)-1
#             stoptime = datetimesUTCnums[stopindex]

        windspdtlist.append(windspds[startindex])
        winddirabstlist.append(winddirabss[startindex])
        fasttemptlist.append(fasttemps[startindex])
        dewpointtlist.append(dewpoints[startindex])
        pressuretlist.append(pressures[startindex])
        windspdavgvectlist.append(windspdsavgvec[startindex])
        winddiravgvectlist.append(winddirsavgvec[startindex])
    
    windspdarr = N.array(windspdtlist)
    winddirabsarr = N.array(winddirabstlist)
    fasttemparr = N.array(fasttemptlist)
    dewpointarr = N.array(dewpointtlist)
    pressurearr = N.array(pressuretlist)
    windspdavgvecarr = N.array(windspdavgvectlist)
    winddiravgvecarr = N.array(winddiravgvectlist)

    windspdlist.append(windspdarr)
    windspdavgveclist.append(windspdavgvecarr)
    winddirabslist.append(winddirabsarr)
    winddiravgveclist.append(winddiravgvecarr)
    fasttemplist.append(fasttemparr)
    dewpointlist.append(dewpointarr)
    pressurelist.append(pressurearr)


In [ ]:
print pressurelist[0]

In [ ]:
radlims = [0.0,250000.,0.,360.]
plotlims = [-30000.,20000.,-40000.,10000.]
ovrmap = False
ovrdis = False
dis_name_list = None
dxy_list = None
fields_D_list = None
deg2rad = N.pi/180.
rlat = radarsweep.latitude['data'][0]
rlon = radarsweep.longitude['data'][0]
print rlat,rlon
print radarsweep.projection


for i,sweeptime in enumerate(sweeptimelist):

    figlist,gridlist = radar.plotsweep_pyART(radlims,plotlims,outfieldnames,
                                             radarsweeplist[i],ovrmap,ovrdis,dis_name_list,dxy_list,fields_D_list)

    ax = gridlist[0][0]

    for j,disfilename in enumerate(disfilenames):
        disfilepath = os.path.join(dis_dir,disfilename)

        GPS_lats,GPS_lons,GPS_stats,GPS_alts,dloc = dis.readPIPSloc(disfilepath)
        dname = disfilename[:6]

        Dx,Dy = pyart.core.geographic_to_cartesian_aeqd(dloc[1],dloc[0],rlon,rlat)
        print Dx,Dy



        stationplot = StationPlot(ax, Dx, Dy)
        stationplot.plot_parameter('NW', [fasttemplist[j][i]], color='red')
        stationplot.plot_parameter('SW', [dewpointlist[j][i]], color='darkgreen')
        stationplot.plot_parameter('NE', [pressurelist[j][i]],
                                   formatter=lambda v: format(10 * v, '.0f')[-3:])

        print windspdavgveclist[j][i],winddiravgveclist[j][i]

    #     u, v = get_wind_components(([windspdavgveclist[i]] * units('m/s')).to('knots'),
    #                                [winddiravgveclist[i]] * units.degree)

        u, v = get_wind_components([windspdavgveclist[j][i]] * units('m/s'),
                                   [winddiravgveclist[j][i]] * units.degree)

        stationplot.plot_barb(u, v, barb_increments=dict(half=0.5, full=1.0, flag=5.0))

    figlist[0].canvas.draw()
    # if(field_D != None):
    #     ax.annotate(disfmtstr.format(field_D),(Dx+250.,Dy-250.),clip_on=True)
    #plt.savefig('/Users/ddawson/temp_VSE/fig{:04d}.png'.format(i),dpi=200)
    #plt.close(figlist[0])
